In [1]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [3]:
from functions import *

In [4]:
def filter_by_sector(tickers_list_,new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date):

    for ticker in tickers_list_:

        if ticker in tickers_remove_list:
            continue

        df_fc = quandl.get_table('ZACKS/FC',ticker = ticker, per_end_date = {'gte':start_date,'lte':end_date},qopts={"columns":"zacks_sector_code"})

        if len(df_fc)!=0:
            sector_code_list = list(df_fc['zacks_sector_code'])

            if bool(set(zacks_sector_code_list) & set(sector_code_list))==False:
                new_ticker_list.append(ticker)

    return new_ticker_list

In [5]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df_tickers = pd.read_csv("zacks-tickers.csv")
tickers_list = list(df_tickers['3262Q'])

zacks_sector_code_list = [5,13]
tickers_remove_list = ['ABEO']

new_ticker_list = []
new_ticker_list = filter_by_sector(tickers_list[1000:2000],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

# new_ticker_list

In [6]:
df_new_ticker2 = pd.DataFrame(new_ticker_list, columns =['ticker'])

In [7]:
df_new_ticker2
    

,ticker
0,BIOAF
1,BIO.B
2,BIOC
3,BIOL
4,BIOQ
...,...
531,CRMT
532,CRMZ
533,CRNCY
534,CRNT


In [8]:
df_new_ticker2.to_csv('filter_by_sector_2000.csv',index=False)

In [9]:
new_ticker_list6 = []
new_ticker_list6 = filter_by_sector(tickers_list[5000:6000],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

df_new_ticker6 = pd.DataFrame(new_ticker_list, columns =['ticker'])
df_new_ticker6.to_csv('filter_by_sector_6000.csv',index=False)

In [10]:
len(df_new_ticker6)

1116

In [4]:
def filter_by_adj_close(tickers_list_,start_date,end_date,num):

    filter_by_adj_close_list = []

    for ticker in tickers_list_:
        df1 = get_adj_close(ticker,start_date,end_date)
        if len(df1) == num:
            filter_by_adj_close_list.append(ticker)

    return filter_by_adj_close_list


start_date = '2014-06-30'
end_date = '2022-06-30'

df6 = pd.read_csv("filter_by_sector_6000.csv")
ticker_list6 = list(df6['ticker'])
print(len(ticker_list6))

filter_by_adj_close_list6 = filter_by_adj_close(ticker_list6,start_date,end_date,2016)

1116


In [5]:
df_new_ticker6 = pd.DataFrame(filter_by_adj_close_list6, columns =['ticker'])
df_new_ticker6.to_csv('filter_by_adj_close_6000.csv',index=False)

In [6]:
len(df_new_ticker6)

444

In [9]:
def filter_by_debt_ratio(df_ticker,start_date,end_date):

    df_ticker['debt_ratio_greater_than_0.1'] = 0

    for i in range(len(df_ticker)):

        ticker = df_ticker['ticker'][i]

        df_zack = generate_zacks_data(ticker,start_date,end_date)
        df_price = get_adj_close(ticker,start_date,end_date)

        df = df_price.join(df_zack,how='outer')
        df['adj_close'] =  df['adj_close'].fillna(method='ffill')

        df1 = debt_to_market_cap(df,df_zack)

        count = df1[['debt_to_market_cap']][df1[['debt_to_market_cap']]  > 0.1].count()

        df_ticker['debt_ratio_greater_than_0.1'][i] = count

        # if i==100:
        #     break

    return df_ticker

In [10]:
df_ticker_adj_close = pd.read_csv("filter_by_adj_close.csv")
print(len(df_ticker_adj_close))

df_ticker_adj_close = df_ticker_adj_close[500:1000]
df_ticker_adj_close.reset_index(inplace=True)
print(len(df_ticker_adj_close))

df_ticker_debt_ratio2 = filter_by_debt_ratio(df_ticker_adj_close,start_date,end_date)

1975
500


In [11]:
df_ticker_debt_ratio2

,index,ticker,debt_ratio_greater_than_0.1
0,500,BTI,0
1,501,BUD,0
2,502,BURL,1084
3,503,BVN,0
4,504,BWEN,1435
...,...,...,...
495,995,DKL,188
496,996,DKS,1469
497,997,DLA,1998
498,998,DLB,0


In [12]:
df_ticker_debt_ratio2.to_csv('filter_by_debt_ratio_2.csv',index=False)